## 필요 라이브러리 설치

!pip install BeautifulSoup4

!pip install pandas

!pip install numpy

## 기본 html 파싱

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
res = requests.get('https://musclewiki.com/Directory')
soup = BeautifulSoup(res.content, 'html.parser')

lst = soup.find_all('tr')

## 필요 데이터 추출

In [3]:
muscle = []
exercise = []
equipment = []
difficulty = []
link = []

for item in lst:
    txt = [tmp for tmp in item.get_text().split('\n') if tmp]
    if 'Exercises' in item.get_text():
        muscle.append(txt[0].strip()[:-10])
        exercise.append([])
        equipment.append([])
        difficulty.append([])
        link.append([])
        
    else:
        exercise[-1].append(txt[0].strip())
        equipment[-1].append(txt[-2].strip())
        difficulty[-1].append(txt[-1].strip())
        tmp = 'https://musclewiki.com' + item.find('a')['href']
        link[-1].append(tmp)

## 데이터프레임 생성

In [6]:
df_lst = []

for i in range(len(muscle)):
    tmp_df = pd.DataFrame([exercise[i], equipment[i], difficulty[i], link[i]], 
                          index = ['exercise', 'equipment', 'difficulty', 'link']).transpose()
    tmp_df['muscle'] = muscle[i]
    df_lst.append(tmp_df)

In [8]:
df_lst[1]

,exercise,equipment,difficulty,link,muscle
0,Barbell Bench Press,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Chest/Barb...,Chest
1,Dips,Bodyweight,Intermediate,https://musclewiki.com/Bodyweight/Male/Chest/Dips,Chest
2,Incline Dumbbell Bench Press,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Chest/In...,Chest
3,Dumbbell Incline Chest Flys,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Chest/Du...,Chest
4,Dumbbell Bench Press,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Chest/Du...,Chest
5,Incline Barbell Bench Press,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Chest/Incl...,Chest
6,Kettlebell Chest Press,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Chest/...,Chest
7,Single Arm Chest Fly,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Chest/...,Chest
8,Single Arm Press,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Chest/...,Chest
9,Walkover Pushup,Kettlebells,Intermediate,https://musclewiki.com/Kettlebells/Male/Chest/...,Chest


## https://www.musclewiki.com + 이미지주소 하면 움짤 주소임!

df = df_lst[0]
a = df.loc[0, 'link']
b = requests.get(a)
s = BeautifulSoup(b.content, 'html.parser')
s.find(class_ = 'exercise-images-grid').find('img')['src']

a = df_lst[4]
r = requests.get(a.loc[0, 'link'])
s = BeautifulSoup(r.content, 'html.parser')
s.get_text()

for df in df_lst:
    for i in range(len(df)):
        tmp_link = df.loc[i, 'link']
        tmp_res = requests.get(tmp_link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = '이미지 주소 오류'
        else:

            images = tmp_soup.find(class_ = 'exercise-images-grid').find_all('img')
            print(i, len(images))
    print()

In [68]:
a = df_lst[0].loc[0, 'link']
r = requests.get(a)
s = BeautifulSoup(r.content, 'html.parser')

i = s.find(class_ = 'exercise-images-grid')
i.find('a')['href']
b = s.find(class_ = 'steps-list')
t = [i for i in b.get_text().split('\n') if i]
g = ""
for i in range(len(t)):
    g = g + str(i + 1) + '. ' + t[i] + '\n'
    
print(g)
    
tmp = df_lst[0]
tmp.loc[0, 'step'] = g
tmp.loc[0, 'step']

1. While holding the upper arms stationary, curl the weights forward while contracting the biceps as you breathe out.
2. Continue the movement until your biceps are fully contracted and the bar is at shoulder level.
3. Hold the contracted position for a second and squeeze the biceps hard.
4. Slowly bring the weight back down to the starting position.



'1. While holding the upper arms stationary, curl the weights forward while contracting the biceps as you breathe out.\n2. Continue the movement until your biceps are fully contracted and the bar is at shoulder level.\n3. Hold the contracted position for a second and squeeze the biceps hard.\n4. Slowly bring the weight back down to the starting position.\n'

In [9]:
for df in df_lst:
    for i in range(len(df)):
        link = df.loc[i, 'link']
        tmp_res = requests.get(link)
        tmp_soup = BeautifulSoup(tmp_res.content, 'html.parser')
        
        if '404' in tmp_soup.get_text():
            df.loc[i, 'img_link'] = 'no data'
            df.loc[i, 'video_link'] = 'no data'
            df.loc[i, 'step'] = 'no data'
            
        else:
            image = tmp_soup.find(class_ = 'exercise-images-grid')
            if image:
                df.loc[i, 'img_link'] = 'https://www.musclewiki.com' + image.find('img')['src']
                df.loc[i, 'video_link'] = image.find('a')['href']
            else:
                df.loc[i, 'img_link'] = 'no data'
                df.loc[i, 'video_link'] = 'no data'
            
            step_parse = tmp_soup.find(class_ = 'steps-list')
            step_lst = [item for item in step_parse.get_text().split('\n') if item]
            step = ""
            for j in range(len(step_lst)):
                step = step + str(j + 1) + '. ' + step_lst[j] + "\n"
            step = step.strip('\n')
            
            if step:
                df.loc[i, 'step'] = step
            else:
                df.loc[i, 'step'] = 'no data'

In [10]:
df_lst[0]

,exercise,equipment,difficulty,link,muscle,img_link,video_link,step
0,Barbell Curl,Barbell,Beginner,https://musclewiki.com/Barbell/Male/Biceps/Bar...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/0mylYdVWBf0,"1. While holding the upper arms stationary, cu..."
1,Dumbbell Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/EU4HlSIpj5Y,1. Stand up straight with a dumbbell in each h...
2,Dumbbell Hammer Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/Dh3_gou_3TE,1. Hold the dumbbells with a neutral grip (thu...
3,Goblet Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/oYmItLMNFg0,1. Stand up straight with a kettlebell in both...
4,Concentration Curl,Kettlebells,Intermediate,https://musclewiki.com/Kettlebells/Male/Biceps...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/xokss8Elgx4,"1. Sitting on a chair with your legs apart, re..."
5,Preacher Curl,Kettlebells,Intermediate,https://musclewiki.com/Kettlebells/Male/Biceps...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/3rN2sSOJV40,"1. Stand behind a bench, with your chest leani..."
6,Single Arm Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/jiheWHzzIJ4,1. Stand up straight with a kettlebell in one ...
7,Biceps Stretch Variation Five,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=7jXK9pNJdJg%7C,1. Stand upright with your feet shoulder width...
8,Biceps Stretch Variation Four,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=IRxy-WjmBRs%7C,1. Stand an arms width away from the wall.\n2....
9,Biceps Stretch Variation Three,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,Biceps,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=koCz9-ZVjRg%7C,"1. Bend your arm at the elbow, raising your ha..."


In [114]:
df = pd.read_csv(path+files[25])
df.loc[13, 'link']
r = requests.get(link)
s = BeautifulSoup(r.content, 'html.parser')
i = s.find(class_ = 'step-list')
if i:
    print('있어')
else:
    print('없어')

없어


df = df_lst[14]
len(df.loc[df['difficulty'] == 'Beginner']) + len(df.loc[df['difficulty'] != 'Beginner'])

df_lst[0].iloc[0]['link']
df_lst[0].loc[0, 'img_front'] = 'hello'
df_lst[0].loc[0, 'img_side'] = 'world'
df_lst[0]

In [46]:
df_lst[1].loc[10, 'step']

'1. Raise your arms to shoulder height, fully extended in front of you.\n2. Slowly bring your arms behind your back still at shoulder height.\n3. Pause for a few seconds and then return to starting position.\n'

In [151]:
beginner = []
master = []

for df in df_lst:
    beginner.append(df.loc[df['difficulty'] == 'Beginner'])
    master.append(df.loc[df['difficulty'] != 'Beginner'])

In [152]:
for i in range(len(df)):
    b_filename = "eng_Beginner_" + muscle[i]
    m_filename = "eng_Master_"+ muscle[i]
    
    b_df = beginner[i].loc[beginner[i]['difficulty'] == 'Beginner']
    m_df = master[i].loc[master[i]['difficulty'] != 'Beginner']
    
    b_df.to_csv('data/' + b_filename + '.csv', index = False)
    m_df.to_csv('data/' + m_filename + '.csv', index = False)

In [50]:
beginner[0]

,exercise,equipment,difficulty,link,step,img_link,video_link
0,Barbell Curl,Barbell,Beginner,https://musclewiki.com/Barbell/Male/Biceps/Bar...,"1. While holding the upper arms stationary, cu...",https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/0mylYdVWBf0
1,Dumbbell Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,1. Stand up straight with a dumbbell in each h...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/EU4HlSIpj5Y
2,Dumbbell Hammer Curl,Dumbbells,Beginner,https://musclewiki.com/Dumbbells/Male/Biceps/D...,1. Hold the dumbbells with a neutral grip (thu...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/Dh3_gou_3TE
3,Goblet Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,1. Stand up straight with a kettlebell in both...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/oYmItLMNFg0
6,Single Arm Curl,Kettlebells,Beginner,https://musclewiki.com/Kettlebells/Male/Biceps...,1. Stand up straight with a kettlebell in one ...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/jiheWHzzIJ4
7,Biceps Stretch Variation Five,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand upright with your feet shoulder width...,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=7jXK9pNJdJg%7C
8,Biceps Stretch Variation Four,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand an arms width away from the wall.\n2....,https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=IRxy-WjmBRs%7C
9,Biceps Stretch Variation Three,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,"1. Bend your arm at the elbow, raising your ha...",https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=koCz9-ZVjRg%7C
10,Biceps Stretch Variation Two,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,"1. Stand facing the wall, an arm's width away....",https://www.musclewiki.com/media/uploads/male-...,https://www.youtube.com/watch?v=_qFHL-DoHG8%7C
11,Biceps Stretch Variation One,Stretches,Beginner,https://musclewiki.com/Stretches/Male/Biceps/B...,1. Stand one foot in front of the other with t...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/KPMttUk2xM8


## 한글로 번역

In [153]:
import os

path = './data/'
files = os.listdir(path)
files = list(filter(lambda x: x[:3] == 'eng', files))

In [165]:
a = pd.read_csv(path + files[25])
files[0].replace('eng', 'kor')

'kor_Beginner_Abdominals.csv'

In [132]:
txt = '데이터 없음'
translator.translate(txt, dest = 'en', src = 'auto').text

'no data'

In [25]:
!pip install googletrans==4.0.0-rc1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.


  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17429 sha256=7d5a51f1c2399a5dcbadbc90c99e14a70d1594e079c98ef2c0ad34e1e9ce6b14
  Stored in directory: c:\users\svbea\appdata\local\pip\cache\wheels\60\b3\27\d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3:
      Successfully uninstalled idna-3.3
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0


In [79]:
import googletrans

In [166]:
translator = googletrans.Translator()

for i in range(len(a)):
    step = a.loc[i, 'step']
    #equipment = a.loc[i, 'equipment']
    #exercise = a.loc[i, 'exercise']
    print(step)
    if step != 'error':
        trans = translator.translate(step, dest = 'ko', src = 'en')
        a.loc[i, 'step'] = trans.text
        print(trans.text)
        
    a.to_csv(path+files[0].replace('eng', 'kor'), encoding = 'utf-8-sig')
    #print(trans.text)
    print()

error

1. Sit on the bench and hold a dumbbell with both hands. Raise the dumbbell overhead at arms length, holding the weight up with the palms of your hands.
2. Keep your elbows in while you lower the weight behind your head, your upper arms stationary.
3. Raise the weight back to starting position.
1. 벤치에 앉아 양손으로 아령을 잡고 있습니다.팔 길이로 아령 오버 헤드를 올려 손바닥으로 무게를 올리십시오.
2. 머리 뒤에 무게를 낮추는 동안 팔꿈치를 유지하십시오.
3. 체중을 다시 시작 위치로 올립니다.

1. Lay on a flat bench while holding a barbell with a shoulder-width grip.
2. Fully extend your elbows until the barbell is directly over your chest.
3. Begin to flex your elbows and allow the barbell to nearly touch your forehead.
4. Extend your elbows back to the starting position and repeat.
1. 어깨 너비 그립이있는 바벨을 들고 평평한 벤치에 놓으십시오.
2. 바벨이 가슴 바로 위에 올 때까지 팔꿈치를 완전히 확장하십시오.
3. 팔꿈치를 구부리기 시작하고 바벨이 이마를 거의 만지도록하십시오.
4. 팔꿈치를 시작 위치로 다시 뻗어 반복하십시오.

1. Lie on your back with knees in the air and soles of your feet flat on the floor.
2. Hold the kettlebell in both hands directly above 

In [161]:
a

,exercise,equipment,difficulty,link,img_link,video_link,step
0,Dips (narrow elbows),Bodyweight,Advanced,https://musclewiki.com/Bodyweight/Male/Triceps...,error,error,error
1,Seated Triceps Extensions,Dumbbells,Intermediate,https://musclewiki.com/Dumbbells/Male/Triceps/...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/_YIMU_LXA6I,1. 벤치에 앉아 양손으로 아령을 잡고 있습니다.팔 길이로 아령 오버 헤드를 올려 ...
2,Laying Triceps Extensions,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Triceps/La...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/hacqeEQXsoo,1. 어깨 너비 그립이있는 바벨을 들고 평평한 벤치에 놓으십시오.\n2. 바벨이 가...
3,Kettlebell Skull Crusher,Kettlebells,Advanced,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kttle...,https://youtu.be/qPRfNvuBy5c,1. 공중에서 무릎을 꿇고 발바닥이 바닥에 평평하게 등을 대고 눕습니다.\n2. 케...
4,Decline Skull Crusher,Kettlebells,Advanced,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kettl...,https://youtu.be/klJfTRXw__U,"1. 윗부분과 발바닥이 바닥에 눕고, 무릎이 구부러지고 골반이 공기에 매달려 있습니..."
5,Tate Press,Kettlebells,Intermediate,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kettl...,https://youtu.be/T3AAwKrp8Uc,1. 등에 평평하게 눕습니다.팔을 똑바로 유지하고 손바닥을 몸에서 멀어지게하여 가슴...
6,Elevated Pike Press,Bodyweight,Intermediate,https://musclewiki.com/Bodyweight/Male/Traps/E...,https://www.musclewiki.com/media/uploads/bodyw...,https://youtu.be/xxWbfDg9oS0,1. 벤치 나 물체를 사용하여 발을 높이십시오.\n2. 팔꿈치를 구부려 머리를 바닥...
7,Dips,Bodyweight,Intermediate,https://musclewiki.com/Bodyweight/Male/Chest/Dips,https://www.musclewiki.com/media/uploads/bodyw...,https://youtu.be/G4BqAiQ9mHs,1. 장비 위에 팔을 잠긴 상태에서 몸을 잡고\n2. 앞으로 기대면서 천천히 몸을 ...
8,Close Grip Bench Press,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Triceps/Cl...,https://www.musclewiki.com/media/uploads/barbe...,https://youtu.be/b5tEUN9jsGY,1. 땅에 발을두고 벤치에 평평하게 놓으십시오.막대에 좁은 그립으로 팔을 똑바로 펴...
9,Cross Pushdown,Cables,Advanced,https://musclewiki.com/Cables/Male/Triceps/Cro...,https://www.musclewiki.com/media/uploads/cable...,https://youtu.be/p2gf2Kj5F7k,1. 두 핸들을 사용하거나 첨부 파일이 전혀 사용되지 않습니다.케이블 크로스 오버의...


In [63]:
pd.read_csv(path+files[25])

,exercise,equipment,difficulty,link,img_link,video_link,step
0,Dips (narrow elbows),Bodyweight,Advanced,https://musclewiki.com/Bodyweight/Male/Triceps...,이미지 주소 오류,NaN,NaN
1,Seated Triceps Extensions,Dumbbells,Intermediate,https://musclewiki.com/Dumbbells/Male/Triceps/...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/_YIMU_LXA6I,1. Sit on the bench and hold a dumbbell with b...
2,Laying Triceps Extensions,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Triceps/La...,https://www.musclewiki.com/media/uploads/male-...,https://youtu.be/hacqeEQXsoo,1. Lay on a flat bench while holding a barbell...
3,Kettlebell Skull Crusher,Kettlebells,Advanced,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kttle...,https://youtu.be/qPRfNvuBy5c,1. Lie on your back with knees in the air and ...
4,Decline Skull Crusher,Kettlebells,Advanced,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kettl...,https://youtu.be/wm5OeiTG2lk,1. Lie on the floor with your upper back and s...
5,Tate Press,Kettlebells,Intermediate,https://musclewiki.com/Kettlebells/Male/Tricep...,https://www.musclewiki.com/media/uploads/kettl...,https://youtu.be/ADSscOmWwmY,1. Lie flat on your back. Keeping your arm str...
6,Elevated Pike Press,Bodyweight,Intermediate,https://musclewiki.com/Bodyweight/Male/Traps/E...,https://www.musclewiki.com/media/uploads/bodyw...,https://youtu.be/xxWbfDg9oS0,1. Use a bench or an object to elevate your fe...
7,Dips,Bodyweight,Intermediate,https://musclewiki.com/Bodyweight/Male/Chest/Dips,https://www.musclewiki.com/media/uploads/bodyw...,https://youtu.be/G4BqAiQ9mHs,1. Hold your body with arms locked above the e...
8,Close Grip Bench Press,Barbell,Intermediate,https://musclewiki.com/Barbell/Male/Triceps/Cl...,https://www.musclewiki.com/media/uploads/barbe...,https://youtu.be/b5tEUN9jsGY,1. Lay flat on the bench with your feet on the...
9,Cross Pushdown,Cables,Advanced,https://musclewiki.com/Cables/Male/Triceps/Cro...,https://www.musclewiki.com/media/uploads/cable...,https://youtu.be/Dv-Rp8gQfmo,1. Either use two handles or no attachment at ...
